<a href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL/blob/master/tutorials/3-Practical/FinRL_PaperTrading_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Disclaimer: Nothing herein is financial advice, and NOT a recommendation to trade real money. Many platforms exist for simulated trading (paper trading) which can be used for building and developing the methods discussed. Please use common sense and always first consult a professional before trading or investing.

# Part 1: Install FinRL

In [ ]:
## install finrl library
!pip install git+https://github.com/AI4Finance-LLC/FinRL-Library.git

## Import related modules

In [4]:
from finrl.train import train
from finrl.test import test
from finrl.config_tickers import DOW_30_TICKER
from finrl.config import INDICATORS
from finrl.finrl.env_stock_trading.env_stocktrading_np import StockTradingEnv
from finrl.finrl.env_stock_trading.env_stock_papertrading import AlpacaPaperTrading
from finrl.finrl.data_processor import DataProcessor
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline

import numpy as np
import pandas as pd

import logging
import sys

from datetime import datetime

logging.basicConfig(
    format='%(asctime)s [%(levelname)s] %(name)s - %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S',
    stream=sys.stdout,
)
logger = logging.getLogger('notebook')

logger.info('Protocol problem: %s', 'connection reset')

# install ipywidgets to fix the jupyter notebook warning
# D:\Anaconda3\envs\finrl\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
#   from .autonotebook import tqdm as notebook_tqdm
# pip install ipywidgets

ModuleNotFoundError: No module named 'finrl.finrl_meta'

## Import Dow Jones 30 Symbols

In [2]:
ticker_list = DOW_30_TICKER
action_dim = len(DOW_30_TICKER)
candle_time_interval = '15Min'

NameError: name 'DOW_30_TICKER' is not defined

In [3]:
print(ticker_list)

NameError: name 'ticker_list' is not defined

In [4]:
print(INDICATORS)

['macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_30_sma', 'close_60_sma']


## Calculate the DRL state dimension manually for paper trading

In [5]:
# amount + (turbulence, turbulence_bool) + (price, shares, cd (holding time)) * stock_dim + tech_dim
state_dim = 1 + 2 + 3 * action_dim + len(INDICATORS) * action_dim

In [1]:
state_dim

NameError: name 'state_dim' is not defined

## Get the API Keys Ready

In [7]:
API_KEY = "PKZOWRCNANQ79J7CV314"
API_SECRET = "7xYTpMGdeffzOsDDsrnl4UryzvJClIpsvZeTIPYf"
API_BASE_URL = 'https://paper-api.alpaca.markets'
data_url = 'wss://data.alpaca.markets'
env = StockTradingEnv

## Show the data

### Step 1. Pick a data source

In [8]:
DP = DataProcessor(data_source = 'alpaca',
                  API_KEY = API_KEY, 
                  API_SECRET = API_SECRET, 
                  API_BASE_URL = API_BASE_URL
                  )

Alpaca successfully connected


### Step 2. Get ticker list, Set start date and end date, specify the data frequency

In [9]:
data = DP.download_data(start_date = '2018-01-01', 
                        end_date = '2022-07-20',
                        ticker_list = ticker_list, 
                        time_interval= candle_time_interval)

2022-07-20 18:57:17 [INFO] notebook - Data before 2018-01-01T15:59:00-05:00 is successfully fetched
2022-07-20 18:57:21 [INFO] notebook - Data before 2018-01-02T15:59:00-05:00 is successfully fetched
2022-07-20 18:57:24 [INFO] notebook - Data before 2018-01-03T15:59:00-05:00 is successfully fetched
2022-07-20 18:57:28 [INFO] notebook - Data before 2018-01-04T15:59:00-05:00 is successfully fetched
2022-07-20 18:57:32 [INFO] notebook - Data before 2018-01-05T15:59:00-05:00 is successfully fetched
2022-07-20 18:57:35 [INFO] notebook - Data before 2018-01-06T15:59:00-05:00 is successfully fetched
2022-07-20 18:57:38 [INFO] notebook - Data before 2018-01-07T15:59:00-05:00 is successfully fetched
2022-07-20 18:57:42 [INFO] notebook - Data before 2018-01-08T15:59:00-05:00 is successfully fetched
2022-07-20 18:57:46 [INFO] notebook - Data before 2018-01-09T15:59:00-05:00 is successfully fetched
2022-07-20 18:57:50 [INFO] notebook - Data before 2018-01-10T15:59:00-05:00 is successfully fetched


2022-07-20 19:01:04 [INFO] notebook - Data before 2018-02-01T15:59:00-05:00 is successfully fetched
2022-07-20 19:01:07 [WARNING] alpaca_trade_api.rest - sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/MSFT/bars 3 more time(s)...
2022-07-20 19:01:12 [INFO] notebook - Data before 2018-02-02T15:59:00-05:00 is successfully fetched
2022-07-20 19:01:12 [WARNING] alpaca_trade_api.rest - sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/GS/bars 3 more time(s)...
2022-07-20 19:01:18 [WARNING] alpaca_trade_api.rest - sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/VZ/bars 3 more time(s)...
2022-07-20 19:01:21 [INFO] notebook - Data before 2018-02-03T15:59:00-05:00 is successfully fetched
2022-07-20 19:01:23 [WARNING] alpaca_trade_api.rest - sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/INTC/bars 3 more time(s)...
2022-07-20 19:01:28 [WARNING] alpaca_trade_api.rest - sleep 3 seconds and retrying https://data.alpaca.marke

2022-07-20 19:04:49 [WARNING] alpaca_trade_api.rest - sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/IBM/bars 3 more time(s)...
2022-07-20 19:04:55 [INFO] notebook - Data before 2018-02-27T15:59:00-05:00 is successfully fetched
2022-07-20 19:04:56 [WARNING] alpaca_trade_api.rest - sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/AXP/bars 3 more time(s)...
2022-07-20 19:05:02 [WARNING] alpaca_trade_api.rest - sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/TRV/bars 3 more time(s)...
2022-07-20 19:05:06 [INFO] notebook - Data before 2018-02-28T15:59:00-05:00 is successfully fetched
2022-07-20 19:05:08 [WARNING] alpaca_trade_api.rest - sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/IBM/bars 3 more time(s)...
2022-07-20 19:05:13 [INFO] notebook - Data before 2018-03-01T15:59:00-05:00 is successfully fetched
2022-07-20 19:05:14 [WARNING] alpaca_trade_api.rest - sleep 3 seconds and retrying https://data.alpaca.marke

2022-07-20 19:08:44 [WARNING] alpaca_trade_api.rest - sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/KO/bars 3 more time(s)...
2022-07-20 19:08:50 [INFO] notebook - Data before 2018-03-25T15:59:00-04:00 is successfully fetched
2022-07-20 19:08:50 [WARNING] alpaca_trade_api.rest - sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/CAT/bars 3 more time(s)...
2022-07-20 19:08:57 [WARNING] alpaca_trade_api.rest - sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/V/bars 3 more time(s)...
2022-07-20 19:09:01 [INFO] notebook - Data before 2018-03-26T15:59:00-04:00 is successfully fetched
2022-07-20 19:09:04 [WARNING] alpaca_trade_api.rest - sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/MSFT/bars 3 more time(s)...
2022-07-20 19:09:08 [INFO] notebook - Data before 2018-03-27T15:59:00-04:00 is successfully fetched
2022-07-20 19:09:11 [WARNING] alpaca_trade_api.rest - sleep 3 seconds and retrying https://data.alpaca.markets

2022-07-20 19:12:53 [WARNING] alpaca_trade_api.rest - sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/AAPL/bars 3 more time(s)...
2022-07-20 19:13:00 [WARNING] alpaca_trade_api.rest - sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/WBA/bars 3 more time(s)...
2022-07-20 19:13:04 [INFO] notebook - Data before 2018-04-22T15:59:00-04:00 is successfully fetched
2022-07-20 19:13:09 [WARNING] alpaca_trade_api.rest - sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/VZ/bars 3 more time(s)...
2022-07-20 19:13:13 [INFO] notebook - Data before 2018-04-23T15:59:00-04:00 is successfully fetched
2022-07-20 19:13:16 [WARNING] alpaca_trade_api.rest - sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/NKE/bars 3 more time(s)...
2022-07-20 19:13:21 [INFO] notebook - Data before 2018-04-24T15:59:00-04:00 is successfully fetched
2022-07-20 19:13:23 [WARNING] alpaca_trade_api.rest - sleep 3 seconds and retrying https://data.alpaca.marke

...

2022-07-20 20:46:44 [INFO] notebook - Data before 2019-11-04T15:59:00-05:00 is successfully fetched
2022-07-20 20:46:57 [INFO] notebook - Data before 2019-11-05T15:59:00-05:00 is successfully fetched
2022-07-20 20:47:09 [INFO] notebook - Data before 2019-11-06T15:59:00-05:00 is successfully fetched
2022-07-20 20:47:22 [INFO] notebook - Data before 2019-11-07T15:59:00-05:00 is successfully fetched
2022-07-20 20:47:35 [INFO] notebook - Data before 2019-11-08T15:59:00-05:00 is successfully fetched
2022-07-20 20:47:47 [INFO] notebook - Data before 2019-11-09T15:59:00-05:00 is successfully fetched
2022-07-20 20:47:59 [INFO] notebook - Data before 2019-11-10T15:59:00-05:00 is successfully fetched
2022-07-20 20:48:12 [INFO] notebook - Data before 2019-11-11T15:59:00-05:00 is successfully fetched
2022-07-20 20:48:25 [INFO] notebook - Data before 2019-11-12T15:59:00-05:00 is successfully fetched
2022-07-20 20:48:37 [INFO] notebook - Data before 2019-11-13T15:59:00-05:00 is successfully fetched


2022-07-20 21:05:01 [INFO] notebook - Data before 2020-01-25T15:59:00-05:00 is successfully fetched
2022-07-20 21:05:32 [INFO] notebook - Data before 2020-01-26T15:59:00-05:00 is successfully fetched
2022-07-20 21:06:35 [INFO] notebook - Data before 2020-01-27T15:59:00-05:00 is successfully fetched
2022-07-20 21:07:02 [INFO] notebook - Data before 2020-01-28T15:59:00-05:00 is successfully fetched
2022-07-20 21:07:16 [INFO] notebook - Data before 2020-01-29T15:59:00-05:00 is successfully fetched
2022-07-20 21:07:31 [INFO] notebook - Data before 2020-01-30T15:59:00-05:00 is successfully fetched
2022-07-20 21:07:45 [INFO] notebook - Data before 2020-01-31T15:59:00-05:00 is successfully fetched
2022-07-20 21:07:58 [INFO] notebook - Data before 2020-02-01T15:59:00-05:00 is successfully fetched
2022-07-20 21:08:12 [INFO] notebook - Data before 2020-02-02T15:59:00-05:00 is successfully fetched
2022-07-20 21:08:27 [INFO] notebook - Data before 2020-02-03T15:59:00-05:00 is successfully fetched


2022-07-20 21:30:35 [INFO] notebook - Data before 2020-04-16T15:59:00-04:00 is successfully fetched
2022-07-20 21:30:51 [INFO] notebook - Data before 2020-04-17T15:59:00-04:00 is successfully fetched
2022-07-20 21:31:05 [INFO] notebook - Data before 2020-04-18T15:59:00-04:00 is successfully fetched
2022-07-20 21:31:19 [INFO] notebook - Data before 2020-04-19T15:59:00-04:00 is successfully fetched
2022-07-20 21:31:35 [INFO] notebook - Data before 2020-04-20T15:59:00-04:00 is successfully fetched
2022-07-20 21:31:49 [INFO] notebook - Data before 2020-04-21T15:59:00-04:00 is successfully fetched
2022-07-20 21:32:05 [INFO] notebook - Data before 2020-04-22T15:59:00-04:00 is successfully fetched
2022-07-20 21:32:20 [INFO] notebook - Data before 2020-04-23T15:59:00-04:00 is successfully fetched
2022-07-20 21:32:35 [INFO] notebook - Data before 2020-04-24T15:59:00-04:00 is successfully fetched
2022-07-20 21:32:52 [INFO] notebook - Data before 2020-04-25T15:59:00-04:00 is successfully fetched


2022-07-20 21:52:03 [INFO] notebook - Data before 2020-07-07T15:59:00-04:00 is successfully fetched
2022-07-20 21:52:19 [INFO] notebook - Data before 2020-07-08T15:59:00-04:00 is successfully fetched
2022-07-20 21:52:35 [INFO] notebook - Data before 2020-07-09T15:59:00-04:00 is successfully fetched
2022-07-20 21:52:52 [INFO] notebook - Data before 2020-07-10T15:59:00-04:00 is successfully fetched
2022-07-20 21:53:10 [INFO] notebook - Data before 2020-07-11T15:59:00-04:00 is successfully fetched
2022-07-20 21:53:25 [INFO] notebook - Data before 2020-07-12T15:59:00-04:00 is successfully fetched
2022-07-20 21:53:42 [INFO] notebook - Data before 2020-07-13T15:59:00-04:00 is successfully fetched
2022-07-20 21:53:58 [INFO] notebook - Data before 2020-07-14T15:59:00-04:00 is successfully fetched
2022-07-20 21:54:14 [INFO] notebook - Data before 2020-07-15T15:59:00-04:00 is successfully fetched
2022-07-20 21:54:30 [INFO] notebook - Data before 2020-07-16T15:59:00-04:00 is successfully fetched


ConnectionError: ('Connection aborted.', ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None))

### Step 3. Data Cleaning & Feature Engineering

In [ ]:
data = DP.clean_data(data)
data = DP.add_technical_indicator(data, INDICATORS)
data = DP.add_vix(data)

In [ ]:
data.tail(20)

### Step 4. Transform to numpy array

In [ ]:
price_array, tech_array, turbulence_array = DP.df_to_array(data, if_vix='True')

In [ ]:
price_array

# Part 2: Train the agent

## Train

In [10]:
ERL_PARAMS = {"learning_rate": 3e-6,"batch_size": 2048,"gamma":  0.985,
        "seed":312,"net_dimension":512, "target_step":5000, "eval_gap":30,
        "eval_times":1} 
#if you want to use larger datasets (change to longer period), and it raises error, 
#please try to increase "target_step". It should be larger than the episode steps. 

PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 128,
}

TRAIN_START_DATE = '2018-01-01'
TRAIN_END_DATE = '2022-07-20'
MODEL_NAME = 'ppo'
TOTAL_TIMESTEPS = 1e6
VERSION= datetime.now().strftime("%Y%m%d-%H%M%S")
TRAIN_CWD = f'./models/papertrading/{MODEL_NAME}_train_{TOTAL_TIMESTEPS}_{TRAIN_START_DATE}_{TRAIN_END_DATE}_{candle_time_interval}_{VERSION}'


In [ ]:

train(start_date = TRAIN_START_DATE, 
      end_date = TRAIN_END_DATE,
      ticker_list = ticker_list, 
      data_source = 'alpaca',
      time_interval= candle_time_interval, 
      technical_indicator_list= INDICATORS,
      drl_lib='stable_baselines3', 
      env=env,
      model_name = MODEL_NAME, 
      API_KEY = API_KEY, 
      API_SECRET = API_SECRET, 
      API_BASE_URL = API_BASE_URL,
      agent_params=PPO_PARAMS,
      total_timesteps=TOTAL_TIMESTEPS,
      cwd=TRAIN_CWD, #current_working_dir
      break_step=1e5)

## Test

In [ ]:
TEST_START_DATE = '2022-07-11'
TEST_END_DATE = '2022-07-19'


In [ ]:
CWD = './models/papertrading/ppo_retrain_1000000.0_2022-01-03_2022-07-18_15Min_1.0.0'
account_value_erl=test(start_date = TEST_START_DATE, 
                      end_date = TEST_END_DATE,
                      ticker_list = ticker_list, 
                      data_source = 'alpaca',
                      time_interval= candle_time_interval, 
                      technical_indicator_list= INDICATORS,
                      drl_lib='stable_baselines3', 
                      env=env, 
                      model_name=MODEL_NAME, 
                      API_KEY = API_KEY, 
                      API_SECRET = API_SECRET, 
                      API_BASE_URL = API_BASE_URL,
                      cwd=CWD,
                      net_dimension = 512)

## Use full data to train 

After tuning well, retrain on the training and testing sets

In [11]:
RETRAIN_START_DATE = '2018-01-01'
RETRAIN_END_DATE = '2022-07-20'
RETRAIN_CWD = f'./models/papertrading/{MODEL_NAME}_retrain_{TOTAL_TIMESTEPS}_{TRAIN_START_DATE}_{TRAIN_END_DATE}_{candle_time_interval}_{VERSION}'


In [12]:
train(start_date = RETRAIN_START_DATE, 
      end_date = RETRAIN_END_DATE,
      ticker_list = ticker_list, 
      data_source = 'alpaca',
      time_interval= candle_time_interval, 
      technical_indicator_list= INDICATORS,
      drl_lib='stable_baselines3', 
      env=env, 
      model_name=MODEL_NAME, 
      API_KEY = API_KEY, 
      API_SECRET = API_SECRET, 
      API_BASE_URL = API_BASE_URL,
      agent_params=PPO_PARAMS,
      total_timesteps=TOTAL_TIMESTEPS,
      cwd=RETRAIN_CWD,
      break_step=5e4)

Alpaca successfully connected
2022-07-20 22:50:54 [INFO] notebook - Data before 2018-01-01T15:59:00-05:00 is successfully fetched
2022-07-20 22:50:54 [INFO] notebook - Data before 2018-01-02T15:59:00-05:00 is successfully fetched
2022-07-20 22:50:54 [INFO] notebook - Data before 2018-01-03T15:59:00-05:00 is successfully fetched
2022-07-20 22:50:54 [INFO] notebook - Data before 2018-01-04T15:59:00-05:00 is successfully fetched
2022-07-20 22:50:55 [INFO] notebook - Data before 2018-01-05T15:59:00-05:00 is successfully fetched
2022-07-20 22:50:55 [INFO] notebook - Data before 2018-01-06T15:59:00-05:00 is successfully fetched
2022-07-20 22:50:55 [INFO] notebook - Data before 2018-01-07T15:59:00-05:00 is successfully fetched
2022-07-20 22:50:55 [INFO] notebook - Data before 2018-01-08T15:59:00-05:00 is successfully fetched
2022-07-20 22:50:55 [INFO] notebook - Data before 2018-01-09T15:59:00-05:00 is successfully fetched
2022-07-20 22:50:56 [INFO] notebook - Data before 2018-01-10T15:59:00-

2022-07-20 22:51:49 [INFO] notebook - Data before 2018-03-24T15:59:00-04:00 is successfully fetched
2022-07-20 22:51:51 [INFO] notebook - Data before 2018-03-25T15:59:00-04:00 is successfully fetched
2022-07-20 22:51:52 [INFO] notebook - Data before 2018-03-26T15:59:00-04:00 is successfully fetched
2022-07-20 22:51:53 [INFO] notebook - Data before 2018-03-27T15:59:00-04:00 is successfully fetched
2022-07-20 22:51:54 [INFO] notebook - Data before 2018-03-28T15:59:00-04:00 is successfully fetched
2022-07-20 22:51:55 [INFO] notebook - Data before 2018-03-29T15:59:00-04:00 is successfully fetched
2022-07-20 22:51:57 [INFO] notebook - Data before 2018-03-30T15:59:00-04:00 is successfully fetched
2022-07-20 22:51:58 [INFO] notebook - Data before 2018-03-31T15:59:00-04:00 is successfully fetched
2022-07-20 22:51:59 [INFO] notebook - Data before 2018-04-01T15:59:00-04:00 is successfully fetched
2022-07-20 22:52:00 [INFO] notebook - Data before 2018-04-02T15:59:00-04:00 is successfully fetched


2022-07-20 22:54:11 [INFO] notebook - Data before 2018-06-14T15:59:00-04:00 is successfully fetched
2022-07-20 22:54:13 [INFO] notebook - Data before 2018-06-15T15:59:00-04:00 is successfully fetched
2022-07-20 22:54:16 [INFO] notebook - Data before 2018-06-16T15:59:00-04:00 is successfully fetched
2022-07-20 22:54:18 [INFO] notebook - Data before 2018-06-17T15:59:00-04:00 is successfully fetched
2022-07-20 22:54:20 [INFO] notebook - Data before 2018-06-18T15:59:00-04:00 is successfully fetched
2022-07-20 22:54:23 [INFO] notebook - Data before 2018-06-19T15:59:00-04:00 is successfully fetched
2022-07-20 22:54:25 [INFO] notebook - Data before 2018-06-20T15:59:00-04:00 is successfully fetched
2022-07-20 22:54:28 [INFO] notebook - Data before 2018-06-21T15:59:00-04:00 is successfully fetched
2022-07-20 22:54:30 [INFO] notebook - Data before 2018-06-22T15:59:00-04:00 is successfully fetched
2022-07-20 22:54:32 [INFO] notebook - Data before 2018-06-23T15:59:00-04:00 is successfully fetched


2022-07-20 22:58:03 [INFO] notebook - Data before 2018-09-04T15:59:00-04:00 is successfully fetched
2022-07-20 22:58:07 [INFO] notebook - Data before 2018-09-05T15:59:00-04:00 is successfully fetched
2022-07-20 22:58:10 [INFO] notebook - Data before 2018-09-06T15:59:00-04:00 is successfully fetched
2022-07-20 22:58:14 [INFO] notebook - Data before 2018-09-07T15:59:00-04:00 is successfully fetched
2022-07-20 22:58:17 [INFO] notebook - Data before 2018-09-08T15:59:00-04:00 is successfully fetched
2022-07-20 22:58:21 [INFO] notebook - Data before 2018-09-09T15:59:00-04:00 is successfully fetched
2022-07-20 22:58:24 [INFO] notebook - Data before 2018-09-10T15:59:00-04:00 is successfully fetched
2022-07-20 22:58:28 [INFO] notebook - Data before 2018-09-11T15:59:00-04:00 is successfully fetched
2022-07-20 22:58:31 [INFO] notebook - Data before 2018-09-12T15:59:00-04:00 is successfully fetched
2022-07-20 22:58:35 [INFO] notebook - Data before 2018-09-13T15:59:00-04:00 is successfully fetched


2022-07-20 23:03:26 [INFO] notebook - Data before 2018-11-25T15:59:00-05:00 is successfully fetched
2022-07-20 23:03:30 [INFO] notebook - Data before 2018-11-26T15:59:00-05:00 is successfully fetched
2022-07-20 23:03:35 [INFO] notebook - Data before 2018-11-27T15:59:00-05:00 is successfully fetched
2022-07-20 23:03:41 [INFO] notebook - Data before 2018-11-28T15:59:00-05:00 is successfully fetched
2022-07-20 23:03:46 [INFO] notebook - Data before 2018-11-29T15:59:00-05:00 is successfully fetched
2022-07-20 23:03:50 [INFO] notebook - Data before 2018-11-30T15:59:00-05:00 is successfully fetched
2022-07-20 23:03:55 [INFO] notebook - Data before 2018-12-01T15:59:00-05:00 is successfully fetched
2022-07-20 23:03:59 [INFO] notebook - Data before 2018-12-02T15:59:00-05:00 is successfully fetched
2022-07-20 23:04:04 [INFO] notebook - Data before 2018-12-03T15:59:00-05:00 is successfully fetched
2022-07-20 23:04:08 [INFO] notebook - Data before 2018-12-04T15:59:00-05:00 is successfully fetched


...

2022-07-21 02:51:23 [INFO] notebook - Data before 2021-10-26T15:59:00-04:00 is successfully fetched
2022-07-21 02:51:45 [INFO] notebook - Data before 2021-10-27T15:59:00-04:00 is successfully fetched
2022-07-21 02:52:07 [INFO] notebook - Data before 2021-10-28T15:59:00-04:00 is successfully fetched
2022-07-21 02:52:29 [INFO] notebook - Data before 2021-10-29T15:59:00-04:00 is successfully fetched
2022-07-21 02:52:50 [INFO] notebook - Data before 2021-10-30T15:59:00-04:00 is successfully fetched
2022-07-21 02:53:12 [INFO] notebook - Data before 2021-10-31T15:59:00-04:00 is successfully fetched
2022-07-21 02:53:34 [INFO] notebook - Data before 2021-11-01T15:59:00-04:00 is successfully fetched
2022-07-21 02:53:57 [INFO] notebook - Data before 2021-11-02T15:59:00-04:00 is successfully fetched
2022-07-21 02:54:18 [INFO] notebook - Data before 2021-11-03T15:59:00-04:00 is successfully fetched
2022-07-21 02:54:41 [INFO] notebook - Data before 2021-11-04T15:59:00-04:00 is successfully fetched


2022-07-21 03:21:39 [INFO] notebook - Data before 2022-01-16T15:59:00-05:00 is successfully fetched
2022-07-21 03:21:59 [INFO] notebook - Data before 2022-01-17T15:59:00-05:00 is successfully fetched
2022-07-21 03:22:19 [INFO] notebook - Data before 2022-01-18T15:59:00-05:00 is successfully fetched
2022-07-21 03:22:40 [INFO] notebook - Data before 2022-01-19T15:59:00-05:00 is successfully fetched
2022-07-21 03:23:03 [INFO] notebook - Data before 2022-01-20T15:59:00-05:00 is successfully fetched
2022-07-21 03:23:24 [INFO] notebook - Data before 2022-01-21T15:59:00-05:00 is successfully fetched
2022-07-21 03:23:44 [INFO] notebook - Data before 2022-01-22T15:59:00-05:00 is successfully fetched
2022-07-21 03:24:03 [INFO] notebook - Data before 2022-01-23T15:59:00-05:00 is successfully fetched
2022-07-21 03:24:23 [INFO] notebook - Data before 2022-01-24T15:59:00-05:00 is successfully fetched
2022-07-21 03:24:43 [INFO] notebook - Data before 2022-01-25T15:59:00-05:00 is successfully fetched


2022-07-21 03:49:45 [INFO] notebook - Data before 2022-04-08T15:59:00-04:00 is successfully fetched
2022-07-21 03:50:05 [INFO] notebook - Data before 2022-04-09T15:59:00-04:00 is successfully fetched
2022-07-21 03:50:26 [INFO] notebook - Data before 2022-04-10T15:59:00-04:00 is successfully fetched
2022-07-21 03:50:47 [INFO] notebook - Data before 2022-04-11T15:59:00-04:00 is successfully fetched
2022-07-21 03:51:08 [INFO] notebook - Data before 2022-04-12T15:59:00-04:00 is successfully fetched
2022-07-21 03:51:29 [INFO] notebook - Data before 2022-04-13T15:59:00-04:00 is successfully fetched
2022-07-21 03:51:49 [INFO] notebook - Data before 2022-04-14T15:59:00-04:00 is successfully fetched
2022-07-21 03:52:10 [INFO] notebook - Data before 2022-04-15T15:59:00-04:00 is successfully fetched
2022-07-21 03:52:31 [INFO] notebook - Data before 2022-04-16T15:59:00-04:00 is successfully fetched
2022-07-21 03:52:51 [INFO] notebook - Data before 2022-04-17T15:59:00-04:00 is successfully fetched


2022-07-21 04:19:19 [INFO] notebook - Data before 2022-06-29T15:59:00-04:00 is successfully fetched
2022-07-21 04:19:41 [INFO] notebook - Data before 2022-06-30T15:59:00-04:00 is successfully fetched
2022-07-21 04:20:03 [INFO] notebook - Data before 2022-07-01T15:59:00-04:00 is successfully fetched
2022-07-21 04:20:24 [INFO] notebook - Data before 2022-07-02T15:59:00-04:00 is successfully fetched
2022-07-21 04:20:46 [INFO] notebook - Data before 2022-07-03T15:59:00-04:00 is successfully fetched
2022-07-21 04:21:07 [INFO] notebook - Data before 2022-07-04T15:59:00-04:00 is successfully fetched
2022-07-21 04:21:29 [INFO] notebook - Data before 2022-07-05T15:59:00-04:00 is successfully fetched
2022-07-21 04:21:51 [INFO] notebook - Data before 2022-07-06T15:59:00-04:00 is successfully fetched
2022-07-21 04:22:13 [INFO] notebook - Data before 2022-07-07T15:59:00-04:00 is successfully fetched
2022-07-21 04:22:36 [INFO] notebook - Data before 2022-07-08T15:59:00-04:00 is successfully fetched


KeyboardInterrupt: 

# Part 3: Deploy the agent

## Setup Alpaca Paper trading environment

In [11]:
import datetime
import threading
from finrl.finrl_meta.data_processors.processor_alpaca import AlpacaProcessor
import alpaca_trade_api as tradeapi
import time
import pandas as pd
import numpy as np
import torch
import gym

class AlpacaPaperTrading():

    def __init__(self,ticker_list, time_interval, drl_lib, agent, cwd, net_dim, 
                 state_dim, action_dim, API_KEY, API_SECRET, 
                 API_BASE_URL, tech_indicator_list, turbulence_thresh=30, 
                 max_stock=1e2, latency = None):
        #load agent
        self.drl_lib = drl_lib
        if agent =='ppo':
            if drl_lib == 'elegantrl':              
                from elegantrl.agents import AgentPPO
                from elegantrl.train.run import init_agent
                from elegantrl.train.config import Arguments
                #load agent
                config = {'state_dim':state_dim,
                            'action_dim':action_dim,}
                args = Arguments(agent=AgentPPO, env=StockEnvEmpty(config))
                args.cwd = cwd
                args.net_dim = net_dim
                # load agent
                try:
                    agent = init_agent(args, gpu_id = 0)
                    self.act = agent.act
                    self.device = agent.device
                except BaseException:
                    raise ValueError("Fail to load agent!")
                        
            elif drl_lib == 'rllib':
                from ray.rllib.agents import ppo
                from ray.rllib.agents.ppo.ppo import PPOTrainer
                
                config = ppo.DEFAULT_CONFIG.copy()
                config['env'] = StockEnvEmpty
                config["log_level"] = "WARN"
                config['env_config'] = {'state_dim':state_dim,
                            'action_dim':action_dim,}
                trainer = PPOTrainer(env=StockEnvEmpty, config=config)
                trainer.restore(cwd)
                try:
                    trainer.restore(cwd)
                    self.agent = trainer
                    print("Restoring from checkpoint path", cwd)
                except:
                    raise ValueError('Fail to load agent!')
                    
            elif drl_lib == 'stable_baselines3':
                from stable_baselines3 import PPO
                
                try:
                    #load agent
                    self.model = PPO.load(cwd)
                    print("Successfully load model", cwd)
                except:
                    raise ValueError('Fail to load agent!')
                    
            else:
                raise ValueError('The DRL library input is NOT supported yet. Please check your input.')
               
        else:
            raise ValueError('Agent input is NOT supported yet.')
            
            
            
        #connect to Alpaca trading API
        try:
            self.alpaca = tradeapi.REST(API_KEY,API_SECRET,API_BASE_URL, 'v2')
        except:
            raise ValueError('Fail to connect Alpaca. Please check account info and internet connection.')
        
        #read trading time interval
        if time_interval == '1s':
            self.time_interval = 1
        elif time_interval == '5s':
            self.time_interval = 5
        elif time_interval == '1Min':
            self.time_interval = 60
        elif time_interval == '5Min':
            self.time_interval = 60 * 5
        elif time_interval == '15Min':
            self.time_interval = 60 * 15
        else:
            raise ValueError('Time interval input is NOT supported yet.')
        
        #read trading settings
        self.tech_indicator_list = tech_indicator_list
        self.turbulence_thresh = turbulence_thresh
        self.max_stock = max_stock 
        
        #initialize account
        self.stocks = np.asarray([0] * len(ticker_list)) #stocks holding
        self.stocks_cd = np.zeros_like(self.stocks) 
        self.cash = None #cash record 
        self.stocks_df = pd.DataFrame(self.stocks, columns=['stocks'], index = ticker_list)
        self.asset_list = []
        self.price = np.asarray([0] * len(ticker_list))
        self.stockUniverse = ticker_list
        self.turbulence_bool = 0
        self.equities = []
        
    def test_latency(self, test_times = 10): 
        total_time = 0
        for i in range(0, test_times):
            time0 = time.time()
            self.get_state()
            time1 = time.time()
            temp_time = time1 - time0
            total_time += temp_time
        latency = total_time/test_times
        print('latency for data processing: ', latency)
        return latency
        
    def run(self):
        orders = self.alpaca.list_orders(status="open")
        for order in orders:
          self.alpaca.cancel_order(order.id)
    
        # Wait for market to open.
        print("Waiting for market to open...")
        tAMO = threading.Thread(target=self.awaitMarketOpen)
        tAMO.start()
        tAMO.join()
        print("Market opened.")
        while True:

          # Figure out when the market will close so we can prepare to sell beforehand.
          clock = self.alpaca.get_clock()
          closingTime = clock.next_close.replace(tzinfo=datetime.timezone.utc).timestamp()
          currTime = clock.timestamp.replace(tzinfo=datetime.timezone.utc).timestamp()
          self.timeToClose = closingTime - currTime
    
          if(self.timeToClose < (60)):
            # Close all positions when 1 minutes til market close.
            print("Market closing soon. Stop trading.")
            #break
            
            # Close all positions when 1 minutes til market close.
            print("Market closing soon.  Closing positions.")
    
            positions = self.alpaca.list_positions()
            for position in positions:
              if(position.side == 'long'):
                orderSide = 'sell'
              else:
                orderSide = 'buy'
              qty = abs(int(float(position.qty)))
              respSO = []
              tSubmitOrder = threading.Thread(target=self.submitOrder(qty, position.symbol, orderSide, respSO))
              tSubmitOrder.start()
              tSubmitOrder.join()
    
            # Run script again after market close for next trading day.
            print("Sleeping until market close (15 minutes).")
            time.sleep(60 * 15)
            
          else:
            trade = threading.Thread(target=self.trade)
            trade.start()
            trade.join()
            last_equity = float(self.alpaca.get_account().last_equity)
            cur_time = time.time()
            self.equities.append([cur_time,last_equity])
            time.sleep(self.time_interval)
            
    def awaitMarketOpen(self):
        isOpen = self.alpaca.get_clock().is_open
        while(not isOpen):
          clock = self.alpaca.get_clock()
          openingTime = clock.next_open.replace(tzinfo=datetime.timezone.utc).timestamp()
          currTime = clock.timestamp.replace(tzinfo=datetime.timezone.utc).timestamp()
          timeToOpen = int((openingTime - currTime) / 60)
          print(str(timeToOpen) + " minutes til market open.")
          time.sleep(60)
          isOpen = self.alpaca.get_clock().is_open
    
    def trade(self):
        state = self.get_state()
        
        if self.drl_lib == 'elegantrl':
            with torch.no_grad():
                s_tensor = torch.as_tensor((state,), device=self.device)
                a_tensor = self.act(s_tensor)  
                action = a_tensor.detach().cpu().numpy()[0]  
                
            action = (action * self.max_stock).astype(int)
            
        elif self.drl_lib == 'rllib':
            action = self.agent.compute_single_action(state)
        
        elif self.drl_lib == 'stable_baselines3':
            action = self.model.predict(state)[0]
            print("action", action)
            
        else:
            raise ValueError('The DRL library input is NOT supported yet. Please check your input.')
        
        self.stocks_cd += 1
        if self.turbulence_bool == 0:
            
#             print('turbulence_bool: ', self.turbulence_bool)
            
            
            
            min_action = 0  # stock_cd
            
#             print('min_action: ', min_action)
#             print('np.where(action < -min_action): ', np.where(action < -min_action))
#             print('np.where(action < -min_action)[0]: ', np.where(action < -min_action)[0])
            
            for index in np.where(action < -min_action)[0]:  # sell_index:
                
#                 print('sell: ')
#                 print('index: ', index)
#                 print('self.stocks[index]: ', self.stocks[index])
                
                sell_num_shares = min(self.stocks[index], -action[index])
                qty =  abs(int(sell_num_shares))
                respSO = []
                tSubmitOrder = threading.Thread(target=self.submitOrder(qty, self.stockUniverse[index], 'sell', respSO))
                tSubmitOrder.start()
                tSubmitOrder.join()
                self.cash = float(self.alpaca.get_account().cash)
                self.stocks_cd[index] = 0

            for index in np.where(action > min_action)[0]:  # buy_index:
                
#                 print('buy: ')
#                 print('index: ', index)
#                 print('self.stocks[index]: ', self.stocks[index])
                
                if self.cash < 0:
                    tmp_cash = 0
                else:
                    tmp_cash = self.cash
                buy_num_shares = min(tmp_cash // self.price[index], abs(int(action[index])))
                qty = abs(int(buy_num_shares))
                respSO = []
                tSubmitOrder = threading.Thread(target=self.submitOrder(qty, self.stockUniverse[index], 'buy', respSO))
                tSubmitOrder.start()
                tSubmitOrder.join()
                self.cash = float(self.alpaca.get_account().cash)
                self.stocks_cd[index] = 0
                
        else:  # sell all when turbulence
            positions = self.alpaca.list_positions()
            for position in positions:
                if(position.side == 'long'):
                    orderSide = 'sell'
                else:
                    orderSide = 'buy'
                qty = abs(int(float(position.qty)))
                respSO = []
                tSubmitOrder = threading.Thread(target=self.submitOrder(qty, position.symbol, orderSide, respSO))
                tSubmitOrder.start()
                tSubmitOrder.join()
            
            self.stocks_cd[:] = 0
            
    
    def get_state(self):
        alpaca = AlpacaProcessor(api=self.alpaca)
        price, tech, turbulence = alpaca.fetch_latest_data(ticker_list = self.stockUniverse, time_interval=candle_time_interval,
                                                     tech_indicator_list=self.tech_indicator_list)
        turbulence_bool = 1 if turbulence >= self.turbulence_thresh else 0
        
        turbulence = (self.sigmoid_sign(turbulence, self.turbulence_thresh) * 2 ** -5).astype(np.float32)
        
        tech = tech * 2 ** -7
        positions = self.alpaca.list_positions()
        stocks = [0] * len(self.stockUniverse)
        for position in positions:
            ind = self.stockUniverse.index(position.symbol)
            stocks[ind] = ( abs(int(float(position.qty))))
        
        stocks = np.asarray(stocks, dtype = float)
        cash = float(self.alpaca.get_account().cash)
        self.cash = cash
        self.stocks = stocks
        self.turbulence_bool = turbulence_bool 
        self.price = price
        
        
        
        amount = np.array(self.cash * (2 ** -12), dtype=np.float32)
        scale = np.array(2 ** -6, dtype=np.float32)
        state = np.hstack((amount,
                    turbulence,
                    self.turbulence_bool,
                    price * scale,
                    self.stocks * scale,
                    self.stocks_cd,
                    tech,
                    )).astype(np.float32)
        print(len(self.stockUniverse))
        return state
        
    def submitOrder(self, qty, stock, side, resp):
        if(qty > 0):
          try:
            self.alpaca.submit_order(stock, qty, side, "market", "day")
            print("Market order of | " + str(qty) + " " + stock + " " + side + " | completed.")
            resp.append(True)
          except:
            print("Order of | " + str(qty) + " " + stock + " " + side + " | did not go through.")
            resp.append(False)
        else:
          print("Quantity is 0, order of | " + str(qty) + " " + stock + " " + side + " | not completed.")
          resp.append(True)

    @staticmethod
    def sigmoid_sign(ary, thresh):
        def sigmoid(x):
            return 1 / (1 + np.exp(-x * np.e)) - 0.5

        return sigmoid(ary / thresh) * thresh
    
class StockEnvEmpty(gym.Env):
    #Empty Env used for loading rllib agent
    def __init__(self,config):
      state_dim = config['state_dim']
      action_dim = config['action_dim']
      self.env_num = 1
      self.max_step = 10000
      self.env_name = 'StockEnvEmpty'
      self.state_dim = state_dim  
      self.action_dim = action_dim
      self.if_discrete = False  
      self.target_return = 9999
      self.observation_space = gym.spaces.Box(low=-3000, high=3000, shape=(state_dim,), dtype=np.float32)
      self.action_space = gym.spaces.Box(low=-1, high=1, shape=(action_dim,), dtype=np.float32)
        
    def reset(self):
        return 

    def step(self, actions):
        return

## Run Paper trading

In [12]:
print(DOW_30_TICKER)

['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']


In [13]:
state_dim

333

In [14]:
action_dim

30

In [ ]:
paper_trading_erl = AlpacaPaperTrading(ticker_list = DOW_30_TICKER, 
                                       time_interval = candle_time_interval, 
                                       drl_lib = 'stable_baselines3', 
                                       agent = 'ppo', 
#                                        cwd = RETRAIN_CWD, 
                                       cwd='models/papertrading/ppo_retrain_1000000.0_2022-01-03_2022-07-19_15Min_20220719-220446',
                                       net_dim = 512, 
                                       state_dim = state_dim, 
                                       action_dim= action_dim, 
                                       API_KEY = API_KEY, 
                                       API_SECRET = API_SECRET, 
                                       API_BASE_URL = API_BASE_URL, 
                                       tech_indicator_list = INDICATORS, 
                                       turbulence_thresh=30, 
                                       max_stock=1e2)
paper_trading_erl.run()

Successfully load model models/papertrading/ppo_retrain_1000000.0_2022-01-03_2022-07-19_15Min_20220719-220446
Waiting for market to open...
Market opened.
2022-07-21 07:44:40 [INFO] notebook - Succesfully add technical indicators
Successfully transformed into array
30


Exception in thread Thread-6:
Traceback (most recent call last):
  File "D:\Anaconda3\envs\finrl\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "D:\Anaconda3\envs\finrl\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "c:\code\forex\finrl\finrl\finrl_meta\env_stock_trading\env_stock_papertrading.py", line 202, in trade
    action = self.model.predict(state)[0]
  File "D:\Anaconda3\envs\finrl\lib\site-packages\stable_baselines3\common\base_class.py", line 562, in predict
    return self.policy.predict(observation, state, episode_start, deterministic)
  File "D:\Anaconda3\envs\finrl\lib\site-packages\stable_baselines3\common\policies.py", line 338, in predict
    actions = self._predict(observation, deterministic=deterministic)
  File "D:\Anaconda3\envs\finrl\lib\site-packages\stable_baselines3\common\policies.py", line 630, in _predict
    return self.get_distribution(observation).get_actions(deterministic=deterministic)
  

2022-07-21 07:59:53 [INFO] notebook - Succesfully add technical indicators
Successfully transformed into array


Exception in thread Thread-7:
Traceback (most recent call last):
  File "D:\Anaconda3\envs\finrl\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "D:\Anaconda3\envs\finrl\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "c:\code\forex\finrl\finrl\finrl_meta\env_stock_trading\env_stock_papertrading.py", line 202, in trade
    action = self.model.predict(state)[0]
  File "D:\Anaconda3\envs\finrl\lib\site-packages\stable_baselines3\common\base_class.py", line 562, in predict
    return self.policy.predict(observation, state, episode_start, deterministic)
  File "D:\Anaconda3\envs\finrl\lib\site-packages\stable_baselines3\common\policies.py", line 338, in predict
    actions = self._predict(observation, deterministic=deterministic)
  File "D:\Anaconda3\envs\finrl\lib\site-packages\stable_baselines3\common\policies.py", line 630, in _predict
    return self.get_distribution(observation).get_actions(deterministic=deterministic)
  

30
2022-07-21 08:15:06 [INFO] notebook - Succesfully add technical indicators
Successfully transformed into array


...

30
2022-07-21 11:17:46 [INFO] notebook - Succesfully add technical indicators
Successfully transformed into array


Exception in thread Thread-20:
Traceback (most recent call last):
  File "D:\Anaconda3\envs\finrl\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "D:\Anaconda3\envs\finrl\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "c:\code\forex\finrl\finrl\finrl_meta\env_stock_trading\env_stock_papertrading.py", line 202, in trade
    action = self.model.predict(state)[0]
  File "D:\Anaconda3\envs\finrl\lib\site-packages\stable_baselines3\common\base_class.py", line 562, in predict
    return self.policy.predict(observation, state, episode_start, deterministic)
  File "D:\Anaconda3\envs\finrl\lib\site-packages\stable_baselines3\common\policies.py", line 338, in predict
    actions = self._predict(observation, deterministic=deterministic)
  File "D:\Anaconda3\envs\finrl\lib\site-packages\stable_baselines3\common\policies.py", line 630, in _predict
    return self.get_distribution(observation).get_actions(deterministic=deterministic)
 

30
2022-07-21 11:32:59 [INFO] notebook - Succesfully add technical indicators
Successfully transformed into array


Exception in thread Thread-21:
Traceback (most recent call last):
  File "D:\Anaconda3\envs\finrl\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "D:\Anaconda3\envs\finrl\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "c:\code\forex\finrl\finrl\finrl_meta\env_stock_trading\env_stock_papertrading.py", line 202, in trade
    action = self.model.predict(state)[0]
  File "D:\Anaconda3\envs\finrl\lib\site-packages\stable_baselines3\common\base_class.py", line 562, in predict
    return self.policy.predict(observation, state, episode_start, deterministic)
  File "D:\Anaconda3\envs\finrl\lib\site-packages\stable_baselines3\common\policies.py", line 338, in predict
    actions = self._predict(observation, deterministic=deterministic)
  File "D:\Anaconda3\envs\finrl\lib\site-packages\stable_baselines3\common\policies.py", line 630, in _predict
    return self.get_distribution(observation).get_actions(deterministic=deterministic)
 

30


# Part 4: Check Portfolio Performance

In [8]:
import alpaca_trade_api as tradeapi
import exchange_calendars as tc
import numpy as np
import pandas as pd
import pytz
import yfinance as yf
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
from datetime import datetime as dt
from finrl.plot import backtest_stats
import matplotlib.pyplot as plt

In [9]:
def get_trading_days(start, end):
    nyse = tc.get_calendar('NYSE')
#     df = nyse.sessions_in_range(pd.Timestamp(start,tz=pytz.UTC),
#                                 pd.Timestamp(end,tz=pytz.UTC))
    df = nyse.sessions_in_range(pd.Timestamp(start),
                                pd.Timestamp(end))
    trading_days = []
    for day in df:
        trading_days.append(str(day)[:10])

    return trading_days

def alpaca_history(key, secret, url, start, end):
    api = tradeapi.REST(key, secret, url, 'v2')
    trading_days = get_trading_days(start, end)
    df = pd.DataFrame()
    for day in trading_days:
#         df = df.append(api.get_portfolio_history(date_start = day,timeframe='5Min').df.iloc[:78])
        df = pd.concat([df, api.get_portfolio_history(date_start = day,timeframe='5Min').df.iloc[:78]])
    equities = df.equity.values
    cumu_returns = equities/equities[0]
    cumu_returns = cumu_returns[~np.isnan(cumu_returns)]
    
    return df, cumu_returns

def DIA_history(start):
    data_df = yf.download(['^DJI'],start=start, interval="5m")
    data_df = data_df.iloc[48:]
    baseline_returns = data_df['Adj Close'].values/data_df['Adj Close'].values[0]
    return data_df, baseline_returns

## Get cumulative return

In [10]:
history_start_date='2022-07-19'
history_end_date='2022-07-20'

df_erl, cumu_erl = alpaca_history(key=API_KEY, 
                                  secret=API_SECRET, 
                                  url=API_BASE_URL, 
                                  start=history_start_date, #must be within 1 month
                                  end='2022-07-20') #change the date if error occurs


In [11]:
df_djia, cumu_djia = DIA_history(start=history_start_date)

[*********************100%***********************]  1 of 1 completed


In [12]:
print(df_erl)


                           profit_loss  profit_loss_pct     equity
timestamp                                                         
2022-07-19 09:30:00-04:00         0.00         0.000000  100000.00
2022-07-19 09:35:00-04:00         0.00         0.000000  100000.00
2022-07-19 09:40:00-04:00         0.00         0.000000  100000.00
2022-07-19 09:45:00-04:00         0.00         0.000000  100000.00
2022-07-19 09:50:00-04:00         0.00         0.000000  100000.00
...                                ...              ...        ...
2022-07-20 15:35:00-04:00       -78.88        -0.000787  100131.16
2022-07-20 15:40:00-04:00       -82.38        -0.000822  100127.66
2022-07-20 15:45:00-04:00       -62.37        -0.000622  100147.67
2022-07-20 15:50:00-04:00       -65.54        -0.000654  100144.50
2022-07-20 15:55:00-04:00       -74.84        -0.000747  100135.20

[156 rows x 3 columns]


In [13]:
print(df_djia)

                                   Open          High           Low  \
Datetime                                                              
2022-07-19 13:30:00-04:00  31630.939453  31653.740234  31630.939453   
2022-07-19 13:35:00-04:00  31653.759766  31699.070312  31652.710938   
2022-07-19 13:40:00-04:00  31699.640625  31724.500000  31698.679688   
2022-07-19 13:45:00-04:00  31724.730469  31727.679688  31710.900391   
2022-07-19 13:50:00-04:00  31711.240234  31718.080078  31700.169922   
...                                 ...           ...           ...   
2022-07-20 15:35:00-04:00  31870.050781  31885.789062  31857.500000   
2022-07-20 15:40:00-04:00  31870.310547  31925.939453  31864.169922   
2022-07-20 15:45:00-04:00  31914.439453  31928.199219  31882.820312   
2022-07-20 15:50:00-04:00  31910.259766  31929.730469  31867.650391   
2022-07-20 15:55:00-04:00  31876.099609  31896.429688  31857.269531   

                                  Close     Adj Close    Volume  
Datetime  

In [14]:
df_erl.tail()

,profit_loss,profit_loss_pct,equity
timestamp,,,
2022-07-20 15:35:00-04:00,-78.88,-0.000787,100131.16
2022-07-20 15:40:00-04:00,-82.38,-0.000822,100127.66
2022-07-20 15:45:00-04:00,-62.37,-0.000622,100147.67
2022-07-20 15:50:00-04:00,-65.54,-0.000654,100144.50
2022-07-20 15:55:00-04:00,-74.84,-0.000747,100135.20


In [15]:
returns_erl = cumu_erl -1 
returns_dia = cumu_djia - 1
returns_dia = returns_dia[:returns_erl.shape[0]]
print('len of erl return: ', returns_erl.shape[0])
print('len of dia return: ', returns_dia.shape[0])

len of erl return:  156
len of dia return:  108


## plot and save

In [16]:
import matplotlib.pyplot as plt
plt.figure(dpi=1000)
plt.grid()
plt.grid(which='minor', axis='y')
plt.title('Stock Trading (Paper trading)', fontsize=20)
plt.plot(returns_erl, label = 'ElegantRL Agent', color = 'red')
#plt.plot(returns_sb3, label = 'Stable-Baselines3 Agent', color = 'blue' )
#plt.plot(returns_rllib, label = 'RLlib Agent', color = 'green')
plt.plot(returns_dia, label = 'DJIA', color = 'grey')
plt.ylabel('Return', fontsize=16)
plt.xlabel('Year 2022', fontsize=16)
plt.xticks(size = 14)
plt.yticks(size = 14)
ax = plt.gca()
ax.xaxis.set_major_locator(ticker.MultipleLocator(78))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(6))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.005))
ax.yaxis.set_major_formatter(ticker.PercentFormatter(xmax=1, decimals=2))
ax.xaxis.set_major_formatter(ticker.FixedFormatter(['','10-19','','10-20',
                                                    '','10-21','','10-22']))
plt.legend(fontsize=10.5)
plt.savefig('papertrading_stock.png')

C:\Users\Super\AppData\Local\Temp\ipykernel_40360\3373249749.py:19: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.xaxis.set_major_formatter(ticker.FixedFormatter(['','10-19','','10-20',
